In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import optuna
from torch.jit import ScriptModule, script_method
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Parameter as TorchParam
from torch import Tensor
from typing import List, Tuple
from joblib import parallel_backend


c:\Users\yuw1si\.conda\envs\my_space\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
p = r'C:\20_Develop\10_Programming\60_thermal-nn\usecase\10_yuping\MeasurementData_Merged_EFAD.csv'

In [4]:
MeasurementData_Merged = pd.read_csv(p, index_col=0)
MeasurementData_Merged.head()

,nEmFild100ms.Rec_10ms_Fild.pp_rbe_CddAgEm_10ms_Fild.rbe_CddRslvr,tqEmFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm,frqSlipFild10ms.Rec_2ms_Fild.pp_rbe_MctAsm_2ms_Fild.rbe_MctAsm,iDaFild10ms.Rec_2ms_Fild.rp_rbe_CddIPha_2ms_Fild.rbe_MctAsm,iQaFild10ms.Rec_2ms_Fild.rp_rbe_CddIPha_2ms_Fild.rbe_MctAsm,uDaFundaFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm,uQaFundaFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm,OilPmpCoolgSpdAct,OilPmpCoolgSpdReq,R_CW_vfCooltEm,...,I_EM_tTMotWinMidLay12Cl06,I_EM_tTMotWinMidLay56Cl06,I_EM_tTMotWinMidLay56Cl12,I_EM_tTMotWinNdeLay2Cl03,I_EM_tTMotWinNdeLay2Cl12,I_EM_tTMotWinNdeLay5Cl03,I_EM_tTMotWinNdeLay5Cl12,Time,profile_id,profile_name
0,0.0,0.0,0.050290,0.875000,0.062500,0.0,0.000000,2999.000000,3000.0,8.0,...,20.921059,NaN,21.653483,19.6,20.2,19.6,13.9,0.175728,1,20250602_112613_EFAD_Si_PS7_EF10_C2_RC02_TqEmM...
1,-1.0,0.0,-0.027593,0.834663,0.000000,0.0,-0.031250,3002.000000,3000.0,8.0,...,20.917589,NaN,21.652054,19.6,20.2,19.6,13.9,0.675728,1,20250602_112613_EFAD_Si_PS7_EF10_C2_RC02_TqEmM...
2,0.0,0.0,0.009212,0.750000,0.062500,0.0,-0.000006,3000.200958,3000.0,8.0,...,20.917004,NaN,21.650740,19.6,20.2,19.6,13.9,1.175728,1,20250602_112613_EFAD_Si_PS7_EF10_C2_RC02_TqEmM...
3,0.0,0.0,-0.046306,0.812500,0.000000,0.0,-0.000017,3000.000000,3000.0,8.0,...,20.916369,NaN,21.647013,19.6,20.2,19.6,13.9,1.675728,1,20250602_112613_EFAD_Si_PS7_EF10_C2_RC02_TqEmM...
4,1.0,0.0,0.074648,0.812500,0.084479,0.0,0.000000,3001.000000,3000.0,8.0,...,20.916947,NaN,21.646527,19.6,20.2,19.6,13.9,2.175728,1,20250602_112613_EFAD_Si_PS7_EF10_C2_RC02_TqEmM...


In [5]:
MeasurementData_Merged['pm'] = MeasurementData_Merged[['ExternTemp.rp_rbe_Cif_10ms_PIExternTemp1.rbe_Cif', 'ExternTemp.rp_rbe_Cif_10ms_PIExternTemp10.rbe_Cif', 'ExternTemp.rp_rbe_Cif_10ms_PIExternTemp5.rbe_Cif','ExternTemp.rp_rbe_Cif_10ms_PIExternTemp7.rbe_Cif']].mean(axis=1)

In [6]:
MeasurementData_Merged['stator_winding'] = MeasurementData_Merged[['I_EM_tTMotWinDeLay2Cl03', 'I_EM_tTMotWinDeLay2Cl12','I_EM_tTMotWinDeLay5Cl03','I_EM_tTMotWinDeLay5Cl06','I_EM_tTMotWinDeLay5Cl12',
                                                                   'I_EM_tTMotWinNdeLay2Cl03','I_EM_tTMotWinNdeLay2Cl12','I_EM_tTMotWinNdeLay5Cl03']].mean(axis=1)

In [7]:
MeasurementData_Merged['Us'] = np.sqrt(MeasurementData_Merged['uDaFundaFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm']**2 + MeasurementData_Merged['uQaFundaFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm']**2)
MeasurementData_Merged['Is'] = np.sqrt(MeasurementData_Merged['iDaFild10ms.Rec_2ms_Fild.rp_rbe_CddIPha_2ms_Fild.rbe_MctAsm']**2 + MeasurementData_Merged['iQaFild10ms.Rec_2ms_Fild.rp_rbe_CddIPha_2ms_Fild.rbe_MctAsm']**2)

In [8]:
input_cols = ['nEmFild100ms.Rec_10ms_Fild.pp_rbe_CddAgEm_10ms_Fild.rbe_CddRslvr',
                'tqEmFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm',
                'iDaFild10ms.Rec_2ms_Fild.rp_rbe_CddIPha_2ms_Fild.rbe_MctAsm',
                'iQaFild10ms.Rec_2ms_Fild.rp_rbe_CddIPha_2ms_Fild.rbe_MctAsm',
                'uDaFundaFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm',
                'uQaFundaFild100ms.Rec_10ms_Fild.pp_rbe_Mct_10ms_Fild.rbe_MctAsm',
                'R_CW_tCooltIvtrOut',
                # 'R_EM_tTMotRshaftOilIn',
                'tEmSnsrFild10ms.Rec_2ms_Fild.rp_rbe_CddTEm_2ms_Fild.rbe_TMdlEm']

In [9]:
data = MeasurementData_Merged.copy()
data = data[data['pm'] <= 200]

In [10]:
data['profile_id']

0         1
1         1
2         1
3         1
4         1
         ..
68341    60
68342    60
68343    60
68344    60
68345    60
Name: profile_id, Length: 68345, dtype: int64

In [11]:
target_cols = ['pm','stator_winding','R_EM_tTMotRshaftOilIn']
temperature_cols = target_cols + ['tEmSnsrFild10ms.Rec_2ms_Fild.rp_rbe_CddTEm_2ms_Fild.rbe_TMdlEm','R_CW_tCooltIvtrOut']
test_profiles = [36, 37]
test_blacklist_profiles = [25]
train_profiles = [p for p in data.profile_id.unique() if p not in test_profiles and p not in test_blacklist_profiles]
profile_sizes = data.groupby("profile_id").agg("size")

In [12]:
device = torch.device("cpu")

In [13]:
data = data.loc[:, input_cols + ["profile_id"] + target_cols].dropna()
data.shape

(68345, 12)

In [14]:
def generate_tensor(profiles_list):
    """Returns profiles of the data set in a coherent 3D tensor with
    time-major shape (T, B, F) where
    T : Maximum profile length
    B : Batch size = Amount of profiles
    F : Amount of input features.

    Also returns a likewise-shaped sample_weights tensor, which zeros out post-padded zeros for use
    in the cost function (i.e., it acts as masking tensor)"""
    tensor = np.full(
        (profile_sizes[profiles_list].max(), len(profiles_list), data.shape[1] - 1),
        np.nan,
    )
    for i, (pid, df) in enumerate(data.loc[data.profile_id.isin(profiles_list), :].groupby("profile_id")):
        assert pid in profiles_list, f"PID is not in {profiles_list}!"
        tensor[:len(df), i, :] = df.drop(columns="profile_id").to_numpy()
        
    sample_weights = 1 - np.isnan(tensor[:, :, 0])
    tensor = np.nan_to_num(tensor).astype(np.float32)
    # tensor = torch.from_numpy(tensor).to(device)
    # sample_weights = torch.from_numpy(sample_weights).to(device)
    return tensor, sample_weights

In [15]:
train_tensor, train_sample_weights = generate_tensor(train_profiles)
test_tensor, test_sample_weights = generate_tensor(test_profiles)

In [16]:
train_tensor.shape, train_sample_weights.shape, test_tensor.shape, test_sample_weights.shape

((4515, 57, 11), (4515, 57), (1127, 2, 11), (1127, 2))

In [17]:
len(train_profiles), len(test_profiles)

(57, 2)

In [20]:
np.save("train_tensor.npy", train_tensor)
np.save("train_sample_weights.npy", train_sample_weights)
np.save("test_tensor.npy", test_tensor)
np.save("test_sample_weights.npy", test_sample_weights)